In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
from pathlib import Path
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
import numpy as np

In [32]:
input_dir = '../data/interim/ecg_runs_ctrl'
output_dir = '../data/interim/ecg_segments_postcovid'

try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

# with open(filepath, 'r') as f:
#     lead_names = f.readlines()[0:12]
files = list( Path(input_dir).glob('*.txt') )
for input_file in files:

    print(input_file)

    a = np.loadtxt(input_file, skiprows=12)
    ecg_signal = a[1,:]  # for now we only use Lead II as it gives the best picture by itself

    try:
        signals, info = nk.ecg_process(ecg_signal, sampling_rate=500, method='neurokit2')
        segments = pd.concat(list( nk.ecg_segment(signals['ECG_Clean'], rpeaks=info['ECG_R_Peaks'], sampling_rate=500).values() ))
        print(f"Found {len(segments['Label'].unique())} segments; Average segment length = {segments.groupby(['Label'])['Signal'].count().mean()}")
        # saveFilePath = os.path.join(output_dir, input_file.name)
        # pd.pivot_table(data=segments, index=segments.index, columns='Label', values='Signal').to_csv(saveFilePath, sep=' ', index=False, header=False)
    except:
        # print(f'Skipped {input_file}')
        continue